In [1]:
import os
from PIL import Image
from sahi.model import MmdetDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction, predict
from sahi.utils.cv import visualize_object_predictions
import numpy as np
import pickle
from collections import defaultdict
import pandas as pd

/vol/bitbucket/jrb21/anaconda3/envs/sodb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load the model

In [2]:
detection_model = MmdetDetectionModel(
    model_path= "models-to-test/latest.pth",
    config_path= "mmdet_configs/xview_tood/tood_crop_300_500_cls_cars_trucks_1e-3_new_pipe.py",
    device='cuda' # or 'cpu'
)

load checkpoint from local path: models-to-test/latest.pth


### Run inference on a single image

In [3]:
test_img_path = "WV3/out_red.tiff"
width, height = (400,400)

sliced_pred_result = get_sliced_prediction(
    test_img_path,
    detection_model, 
    slice_width=width, 
    slice_height=height
)

Performing prediction on 115 number of slices.


In [4]:
sliced_pred_result.object_prediction_list

[ObjectPrediction<
     bbox: BoundingBox: <(666, 6302, 676, 6319), w: 10, h: 17>,
     mask: None,
     score: PredictionScore: <value: 0.5692824721336365>,
     category: Category: <id: 0, name: Small Car>>,
 ObjectPrediction<
     bbox: BoundingBox: <(680, 6687, 689, 6702), w: 9, h: 15>,
     mask: None,
     score: PredictionScore: <value: 0.5174318552017212>,
     category: Category: <id: 0, name: Small Car>>,
 ObjectPrediction<
     bbox: BoundingBox: <(649, 3191, 658, 3207), w: 9, h: 16>,
     mask: None,
     score: PredictionScore: <value: 0.5075268149375916>,
     category: Category: <id: 0, name: Small Car>>,
 ObjectPrediction<
     bbox: BoundingBox: <(505, 4882, 515, 4898), w: 10, h: 16>,
     mask: None,
     score: PredictionScore: <value: 0.5052633881568909>,
     category: Category: <id: 0, name: Small Car>>,
 ObjectPrediction<
     bbox: BoundingBox: <(489, 5068, 498, 5085), w: 9, h: 17>,
     mask: None,
     score: PredictionScore: <value: 0.500849723815918>,
     c

### Batch prediction

In [98]:
MODEL_PATH = "models-to-test/latest.pth"
MODEL_CONFIG_PATH = config_path= "mmdet_configs/xview_tood/tood_crop_300_500_cls_cars_trucks_1e-3_new_pipe.py"
EVAL_IMAGES_FOLDER_DIR = "WV3"
EXPORT_VISUAL = True

INFERENCE_SETTING_TO_PARAMS = {
    "XVIEW_SAHI": {
        "no_standard_prediction": True,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0,
    },
    "XVIEW_SAHI_PO": {
        "no_standard_prediction": True,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0.20,
    },
    "XVIEW_SAHI_FI": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0,
    },
    "XVIEW_SAHI_FI_PO": {
        "no_standard_prediction": False,
        "no_sliced_prediction": False,
        "slice_size": 400,
        "overlap_ratio": 0.20,
    },
}

# Of the above 4 options I want to slice and not do full image size inference
INFERENCE_SETTING = "XVIEW_SAHI_PO"
setting_params = INFERENCE_SETTING_TO_PARAMS[INFERENCE_SETTING]

# From the evaluation py file I delete the eval dataset path and change the confidence to 0.3
result = predict(
    model_type="mmdet",
    model_path=MODEL_PATH,
    model_config_path=MODEL_CONFIG_PATH,
    model_confidence_threshold=0.25,
    model_device="cuda:0",
    model_category_mapping=None,
    model_category_remapping=None,
    source=EVAL_IMAGES_FOLDER_DIR,
    no_standard_prediction=setting_params["no_standard_prediction"],
    no_sliced_prediction=setting_params["no_sliced_prediction"],
    image_size=None,
    slice_height=setting_params["slice_size"],
    slice_width=setting_params["slice_size"],
    overlap_height_ratio=setting_params["overlap_ratio"],
    overlap_width_ratio=setting_params["overlap_ratio"],
    postprocess_type="GREEDYNMM",
    postprocess_match_metric="IOS",
    postprocess_match_threshold=0.5,
    postprocess_class_agnostic=True,
    novisual=not EXPORT_VISUAL,
    project="runs/highways",
    name=INFERENCE_SETTING,
    visual_bbox_thickness=1,
    visual_text_size=0.3,
    visual_text_thickness=1,
    visual_export_format="png",
    verbose=1,
    return_dict=True,
    force_postprocess_type=True,
    export_pickle = True,
)

There are 4 listed files in folder: WV3/
load checkpoint from local path: models-to-test/latest.pth


Performing inference on images:   0%|                                                                                                 | 0/4 [00:00<?, ?it/s]

Performing prediction on 72 number of slices.


Performing inference on images:  25%|██████████████████████▎                                                                  | 1/4 [00:05<00:17,  5.69s/it]

Prediction time is: 5492.86 ms


...

Performing prediction on 224 number of slices.


Performing inference on images:  75%|██████████████████████████████████████████████████████████████████▊                      | 3/4 [00:38<00:07,  7.34s/it]

Prediction time is: 16830.29 ms


Performing inference on images: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:38<00:00,  9.68s/it]

Prediction results are successfully exported to runs/highways/XVIEW_SAHI_PO10


In [22]:
pickle_folder = 'runs/highways/XVIEW_SAHI_PO9/pickles'

for i, file in enumerate(os.listdir(pickle_folder)):
    with (open(os.path.join(pickle_folder, file), "rb")) as f:
        prediction_list = pickle.load(f)

In [80]:
min([item.score.value for item in prediction_list])

0.2546004354953766

In [81]:
len(prediction_list)

92

In [39]:
clean_train_report_df = pd.read_feather('/vol/bitbucket/jrb21/project/Highways/clean_train_report_df.feather')

In [75]:
M42_sites_southbound = ['10560', '10337']
M42_sites_northbound = ['10398']
M42_sites  = M42_sites_northbound + M42_sites_southbound

In [53]:
clean_train_report_df.loc[(clean_train_report_df.site_id.isin(M42_sites_southbound)) & (clean_train_report_df.DOW=='1')]

,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,total_volume,timestamp,hour,DOW,month,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
570481,M42/6450M,10337,2021-06-01T00:00:00,00:14:00,0,4,3,1,4,61,12,2021-06-01 00:14:00,0,1,6,0.122612,0.185574,0.088146,0.199403,0.149621
570482,M42/6450M,10337,2021-06-01T00:00:00,00:29:00,1,7,2,1,13,58,23,2021-06-01 00:29:00,0,1,6,0.214571,0.123716,0.088146,0.648058,0.286773
570483,M42/6450M,10337,2021-06-01T00:00:00,00:44:00,2,5,3,2,5,60,15,2021-06-01 00:44:00,0,1,6,0.153265,0.185574,0.176292,0.249253,0.187026
570484,M42/6450M,10337,2021-06-01T00:00:00,00:59:00,3,3,2,2,9,58,16,2021-06-01 00:59:00,0,1,6,0.091959,0.123716,0.176292,0.448656,0.199494
570485,M42/6450M,10337,2021-06-01T00:00:00,01:14:00,4,6,1,0,7,62,14,2021-06-01 01:14:00,1,1,6,0.183918,0.061858,0.000000,0.348954,0.174558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778526,M42/6450B,10560,2022-06-14T00:00:00,22:59:00,91,184,36,24,48,63,292,2022-06-14 22:59:00,22,1,6,0.456606,0.341124,0.441301,0.654563,0.458962
778527,M42/6450B,10560,2022-06-14T00:00:00,23:14:00,92,188,35,10,39,64,272,2022-06-14 23:14:00,23,1,6,0.466532,0.331648,0.183875,0.531832,0.427526
778528,M42/6450B,10560,2022-06-14T00:00:00,23:29:00,93,137,25,16,56,63,234,2022-06-14 23:29:00,23,1,6,0.339973,0.236892,0.294201,0.763656,0.367798
778529,M42/6450B,10560,2022-06-14T00:00:00,23:44:00,94,73,22,13,55,60,163,2022-06-14 23:44:00,23,1,6,0.181153,0.208465,0.239038,0.750020,0.256201


In [70]:
clean_train_report_df.timestamp.dt.day.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [76]:
clean_train_report_df.loc[(clean_train_report_df.timestamp.dt.year==2021) &
                         (clean_train_report_df.timestamp.dt.month==6) & 
                         (clean_train_report_df.timestamp.dt.day==15) &
                          (clean_train_report_df.timestamp.dt.hour==11) &
                         (clean_train_report_df.site_id.isin(M42_sites_northbound))]

,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,total_volume,timestamp,hour,DOW,month,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
654908,M42/6438A,10398,2021-06-15T00:00:00,11:14:00,44,438,139,136,188,59,901,2021-06-15 11:14:00,11,1,6,1.020019,1.552031,1.965836,2.012578,1.321995
654909,M42/6438A,10398,2021-06-15T00:00:00,11:29:00,45,473,131,123,182,59,909,2021-06-15 11:29:00,11,1,6,1.101528,1.462706,1.777925,1.948347,1.333734
654910,M42/6438A,10398,2021-06-15T00:00:00,11:44:00,46,445,98,131,211,58,885,2021-06-15 11:44:00,11,1,6,1.036321,1.094238,1.893562,2.258798,1.298519
654911,M42/6438A,10398,2021-06-15T00:00:00,11:59:00,47,478,128,138,204,58,948,2021-06-15 11:59:00,11,1,6,1.113172,1.429209,1.994745,2.183861,1.390956


In [97]:
clean_train_report_df.loc[(clean_train_report_df.timestamp.dt.year==2021) &
                         (clean_train_report_df.timestamp.dt.month==6) & 
                         (clean_train_report_df.timestamp.dt.day==15) &
                          (clean_train_report_df.timestamp.dt.hour==11) &
                         (clean_train_report_df.site_id.isin(M42_sites_southbound))]

,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,total_volume,timestamp,hour,DOW,month,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
571869,M42/6450M,10337,2021-06-15T00:00:00,11:14:00,44,45,30,27,26,58,128,2021-06-15 11:14:00,11,1,6,1.379382,1.855742,2.379946,1.296117,1.595956
571870,M42/6450M,10337,2021-06-15T00:00:00,11:29:00,45,29,17,23,48,55,117,2021-06-15 11:29:00,11,1,6,0.888935,1.051587,2.027361,2.392831,1.458803
571871,M42/6450M,10337,2021-06-15T00:00:00,11:44:00,46,28,24,17,40,57,109,2021-06-15 11:44:00,11,1,6,0.858282,1.484594,1.498484,1.994026,1.359056
571872,M42/6450M,10337,2021-06-15T00:00:00,11:59:00,47,38,21,17,44,57,120,2021-06-15 11:59:00,11,1,6,1.164811,1.299020,1.498484,2.193428,1.496209
746372,M42/6450B,10560,2021-06-15T00:00:00,11:14:00,44,433,152,84,123,63,792,2021-06-15 11:14:00,11,1,6,1.074513,1.440301,1.544554,1.677316,1.244855
746373,M42/6450B,10560,2021-06-15T00:00:00,11:29:00,45,476,146,95,130,62,847,2021-06-15 11:29:00,11,1,6,1.181220,1.383447,1.746817,1.772773,1.331303
746374,M42/6450B,10560,2021-06-15T00:00:00,11:44:00,46,475,153,98,123,63,849,2021-06-15 11:44:00,11,1,6,1.178738,1.449777,1.801980,1.677316,1.334447
746375,M42/6450B,10560,2021-06-15T00:00:00,11:59:00,47,495,139,77,109,63,820,2021-06-15 11:59:00,11,1,6,1.228369,1.317118,1.415841,1.486402,1.288865


In [96]:
clean_train_report_df.loc[(clean_train_report_df.timestamp.dt.year==2021) &
                         (clean_train_report_df.timestamp.dt.month==6) & 
                         (clean_train_report_df.timestamp.dt.day==15) &
                          (clean_train_report_df.timestamp.dt.hour==11) &
                         (clean_train_report_df.site_id == '10337')]

,site_name,site_id,report_date,time_period_ending,time_interval,0-520cm,521-660cm,661-1160cm,1160+cm,avg_mph,total_volume,timestamp,hour,DOW,month,0-520cm_normalised,521-660cm_normalised,661-1160cm_normalised,1160+cm_normalised,total_volume_normalised
571869,M42/6450M,10337,2021-06-15T00:00:00,11:14:00,44,45,30,27,26,58,128,2021-06-15 11:14:00,11,1,6,1.379382,1.855742,2.379946,1.296117,1.595956
571870,M42/6450M,10337,2021-06-15T00:00:00,11:29:00,45,29,17,23,48,55,117,2021-06-15 11:29:00,11,1,6,0.888935,1.051587,2.027361,2.392831,1.458803
571871,M42/6450M,10337,2021-06-15T00:00:00,11:44:00,46,28,24,17,40,57,109,2021-06-15 11:44:00,11,1,6,0.858282,1.484594,1.498484,1.994026,1.359056
571872,M42/6450M,10337,2021-06-15T00:00:00,11:59:00,47,38,21,17,44,57,120,2021-06-15 11:59:00,11,1,6,1.164811,1.299020,1.498484,2.193428,1.496209


In [90]:
road_len_m = 4307.1/2 
road_len_km = road_len_m/1000
road_len_miles = 5/8 * road_len_km

avg_speed_mph = 59

# time = distance/speed
time_to_cross_hours = road_len_miles/avg_speed_mph

time_to_cross_min = time_to_cross_hours * 60

time_to_cross_min

1.3687817796610169

In [92]:
len(prediction_list) * 15/time_to_cross_min

1008.1957697754871